# Experiments

Just to be clear, the following examples are for demonstration and testing purposes only. Users are not supposed to make their own dummy batches using their own data. Refer to the pages **High-Level API** and/ or **Low-Level API** if you actually want to use the library. 

## Prerequisites
* If you've already completed the instructions on the **Installation** page, then let's get started.
* This section makes use of the data in the **Example Datasets** page.

In [1]:
import os 
os.chdir('/Users/Layne/Desktop/aiqc')

In [2]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Dummy Experiments

The `list_demo_batches()` method describes the predefined experiments that are available.

> By default it returns a Pandas DataFrame, but you can `list_demo_files(format='list')` to change that.

In [3]:
examples.list_demo_batches()

,batch_name,supervision,analysis,sub_analysis,validation,fileset
0,multiclass,supervised,classification,multi label,validation split,iris.tsv
1,binary,supervised,classification,binary,validation split,sonar.csv
2,regression,supervised,regression,None,validation split,houses.csv
3,multiclass_folded,supervised,classification,multi label,cross-folds,iris_10x.tsv


* Each `Batch` contains predefined training `Jobs` and a `Hyperparamset`.
  * Each `Job` is comprised of:
    * An `Algorithm` and a unique `Hyperparamcombo`.
    * They gets fit to a `Datasets` made from a dummy data file seen above.
      * More specifically, its `Labels, Featuresets` are sliced into `Splitsets, Foldsets`.

## Example: supervised regression

The entries from the `name` column above, can be fed into the `make_demo_batch()` method.

In [4]:
batch = examples.make_demo_batch('regression')

### ORM in action

It's easy to dig into the Batches using the ORM to see how they are constructed.

In [5]:
batch.job_count

4

In [6]:
batch.jobs[3].hyperparamcombo.hyperparameters

{'neuron_count': 48, 'epochs': 75}

In [7]:
batch.splitset.featureset.dataset.files[0].file_format

'parquet'

In [8]:
batch.algorithm.function_model_train

<function aiqc.examples.regression_model_train(model, samples_train, samples_evaluate, **hyperparameters)>

### Hyperparameter tuning in action

When you execute a Batch of Jobs, it will run the jobs on a background process, so you can continue to work in your Python shell.

> It does this by spawning a `multiprocessing.Process` that lives until each Job is completed or the process is killed.

In [12]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 4/4 [00:24<00:00,  6.20s/it]


The execution of the Jobs can easily be stopped, and then resumed at the Job that was previously running.

In [ ]:
batch.stop_jobs()

In [ ]:
batch.run_jobs()

### Inspecting the Result of an Individual Job

When each Job completes, it will create a `Result` that contains `predictions` and `metrics` attributes for each split/ fold.

In [13]:
batch.jobs[3].results[0].metrics

{'test': {'r2': 0.8016676929231614,
  'mse': 0.16301016148458597,
  'explained_variance': 0.8016679921070664,
  'loss': 0.1630101501941681},
 'validation': {'r2': 0.8064215016436613,
  'mse': 0.18897737807890366,
  'explained_variance': 0.8072795194658497,
  'loss': 0.18897737562656403},
 'train': {'r2': 0.7864009292051323,
  'mse': 0.2135990707948677,
  'explained_variance': 0.7865304609905488,
  'loss': 0.2135990709066391}}

The JSON values of these attributes are interoperable with Python dictionaries.

In [14]:
batch.jobs[3].results[0].predictions['validation']

array([[-0.68978953],
       [-0.14231452],
       [ 0.06651115],
       [ 0.23199542],
       [ 0.41630107],
       [ 0.79887897],
       [ 1.4871986 ],
       [-0.38264734],
       [-0.6172692 ],
       [ 0.05014512],
       [-0.51803964],
       [ 0.3792256 ],
       [ 0.09406288],
       [-0.19306827],
       [ 1.8847973 ],
       [-0.00461408],
       [ 0.31235802],
       [-0.5619943 ],
       [ 0.09901989],
       [ 1.659003  ],
       [ 1.9264379 ],
       [-0.10228682],
       [-0.41020823],
       [-0.8134756 ],
       [ 0.15139169],
       [ 0.66028714],
       [ 0.35218498],
       [-0.774425  ],
       [ 0.2935084 ],
       [ 0.1254541 ],
       [-0.65894353],
       [-0.17424902],
       [ 0.06840625],
       [ 1.5314872 ],
       [ 0.19186011],
       [ 0.7086599 ],
       [-0.12144318],
       [ 0.93836665],
       [-0.35432696],
       [ 0.00953457],
       [-0.70820546],
       [-1.2797608 ],
       [-0.08237875],
       [-0.7953497 ],
       [-0.37204528],
       [-0

### Inspecting the metrics of the entire Batch

In [15]:
batch.metrics_to_pandas()

,job_id,split,r2,mse,explained_variance,loss
0,4,test,0.763881,0.194067,0.763940,0.194067
1,4,validation,0.756629,0.237587,0.757139,0.237587
2,4,train,0.744731,0.255269,0.745199,0.255269
3,5,test,0.782874,0.178456,0.791384,0.178456
4,5,validation,0.790559,0.204463,0.794467,0.204463
5,5,train,0.769850,0.230150,0.774275,0.230150
6,6,test,0.756108,0.200456,0.765129,0.200456
7,6,validation,0.773478,0.221138,0.775778,0.221138
8,6,train,0.755629,0.244371,0.759163,0.244371
9,7,test,0.801668,0.163010,0.801668,0.163010


## Example: supervised multi-classification with cross-fold validation

The following method uses *demo_file* 'iris_10x.tsv', so if you see duplicate samples that is why.

In [16]:
batch = examples.make_demo_batch('multiclass_folded')

In [17]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|████████████████████████████████████████| 10/10 [00:51<00:00,  5.16s/it]


In [18]:
batch.job_count

10

In [20]:
batch.metrics_to_pandas()

,job_id,split,roc_auc,accuracy,precision,recall,f1,loss
0,8,test,0.999570,0.933333,0.944444,0.933333,0.932660,0.145964
1,8,folds_train_combined,0.998997,0.939286,0.948313,0.939286,0.938706,0.134224
2,8,fold_validation,0.999416,0.942857,0.952653,0.942857,0.942771,0.126636


### API Details

For more detailed information reference the **Low-Level API** documentation.

### Metrics & Visualization

For more information of visualization of performance metrics, reference the **Visualization & Metrics** documentation.